In [13]:
from cleanvision import Imagelab
from pathlib import Path

In [14]:
data_path = Path("D:/Projects/DL/MonumentDetection/augmentedImages")

In [15]:
imagelab = Imagelab(data_path)

Reading images from D:/Projects/DL/MonumentDetection/augmentedImages


In [16]:
imagelab.find_issues()

Checking for dark, light, odd_aspect_ratio, low_information, exact_duplicates, near_duplicates, blurry, grayscale, odd_size images ...


  0%|          | 0/4358 [00:00<?, ?it/s]

  0%|          | 0/4358 [00:00<?, ?it/s]

Issue checks completed. 0 issues found in the dataset. To see a detailed report of issues found, use imagelab.report().


In [17]:
imagelab.report()

Issues found in images in order of severity in the dataset

|    | issue_type       |   num_images |
|---:|:-----------------|-------------:|
|  0 | dark             |            0 |
|  1 | light            |            0 |
|  2 | odd_aspect_ratio |            0 |
|  3 | low_information  |            0 |
|  4 | blurry           |            0 |
|  5 | grayscale        |            0 |
|  6 | odd_size         |            0 |
|  7 | exact_duplicates |            0 |
|  8 | near_duplicates  |            0 | 

